In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!unzip '/content/drive/MyDrive/LukaKu/Dataset/Dataset_CAPSTONE.zip'

Streaming output truncated to the last 5000 lines.
  inflating: content/img/pressure-ulcer-images/1355.jpg  
  inflating: content/img/pressure-ulcer-images/1425.jpg  
  inflating: content/img/pressure-ulcer-images/1353.jpg  
  inflating: content/img/pressure-ulcer-images/0826.jpg  
  inflating: content/img/pressure-ulcer-images/1443.jpg  
  inflating: content/img/pressure-ulcer-images/pressure-ulcer-on-foot-0033.jpg  
  inflating: content/img/pressure-ulcer-images/treating--pressure-ulcers0001.jpg  
  inflating: content/img/pressure-ulcer-images/1381_cut.jpg  
  inflating: content/img/pressure-ulcer-images/0974.jpg  
  inflating: content/img/pressure-ulcer-images/1093_cut.jpg  
  inflating: content/img/pressure-ulcer-images/0853.jpg  
  inflating: content/img/pressure-ulcer-images/1279.jpg  
  inflating: content/img/pressure-ulcer-images/1185_cut.jpg  
  inflating: content/img/pressure-ulcer-images/1208_cut.jpg  
  inflating: content/img/pressure-ulcer-images/1263.jpg  
  inflating: co

In [7]:
!pip install split-folders

In [8]:
import os 

dir_i = os.listdir('/content/content/img')

for i in dir_i:
  dir_a = os.path.join('/content/content/img', i)
  print(i, ' has ', len(os.listdir(dir_a)), 'images')

leg-ulcer-images  has  134 images
pilonidal-sinus  has  3 images
skin tears  has  54 images
meningitis  has  24 images
foot-ulcers  has  814 images
normal  has  300 images
DFU  has  1163 images
abdominal-wounds  has  13 images
toes  has  35 images
malignant-wound-images  has  27 images
epidermolysis-bullosa  has  5 images
venous  has  468 images
orthopaedic wounds  has  49 images
surgical  has  634 images
pressure-ulcer-images  has  1221 images
haemangioma  has  6 images
extravasation-wound-images  has  20 images
miscellaneous  has  56 images
burns  has  714 images
trauma  has  103 images


In [9]:
import splitfolders

folder = '/content/content/img'

splitfolders.ratio(folder, output='/content/Dataset_split',
                 seed=42, ratio=(0.9, 0.1),
                 group_prefix = None
                 )

Copying files: 5843 files [00:00, 6525.08 files/s]


In [10]:
import os 

dir_i = os.listdir('/content/Dataset_split/train')

for i in dir_i:
  dir_a = os.path.join('/content/Dataset_split/train', i)
  print(i, ' has ', len(os.listdir(dir_a)), 'data')

leg-ulcer-images  has  120 data
pilonidal-sinus  has  2 data
skin tears  has  48 data
meningitis  has  21 data
foot-ulcers  has  732 data
normal  has  270 data
DFU  has  1046 data
abdominal-wounds  has  11 data
toes  has  31 data
malignant-wound-images  has  24 data
epidermolysis-bullosa  has  4 data
venous  has  421 data
orthopaedic wounds  has  44 data
surgical  has  570 data
pressure-ulcer-images  has  1098 data
haemangioma  has  5 data
extravasation-wound-images  has  18 data
miscellaneous  has  50 data
burns  has  642 data
trauma  has  92 data


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/Dataset_split/train'
val_dir = '/content/Dataset_split/val'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest',
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
)

In [21]:
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2

tf.device('/device:GPU:0')

model = tf.keras.models.Sequential([
    ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    #tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense((20), activation='softmax')  
])
model.layers[0].trainable = False

In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 5, 5, 2048)        58331648  
                                                                 
 flatten_3 (Flatten)         (None, 51200)             0         
                                                                 
 dense_8 (Dense)             (None, 512)               26214912  
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_9 (Dense)             (None, 20)                10260     
                                                                 
Total params: 84,556,820
Trainable params: 26,225,172
Non-trainable params: 58,331,648
_________________________________________________________________


In [23]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical',
    ) # set as training data
validation_generator = train_datagen.flow_from_directory(
    val_dir, # same directory as training data
    target_size=(150, 150),
    batch_size=64,
    class_mode='categorical',
    )

Found 5249 images belonging to 20 classes.
Found 594 images belonging to 20 classes.


In [24]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [25]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.99 and logs.get('val_accuracy')>0.99):
      self.model.stop_training = True
      print("\nThe accuracy of the training set and the validation set has reached > 99%!")
callbacks = myCallback()

In [26]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

In [27]:
mc = ModelCheckpoint('best_model_3_v4.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
# fitting model
history = model.fit(
    train_generator,
    epochs = 30,
    validation_data = validation_generator,
    verbose = 1,
    callbacks=[callbacks, es, mc]
)

Epoch 1/30
83/83 [==============================] - ETA: 0s - loss: 8.0405 - accuracy: 0.3368
Epoch 1: val_accuracy improved from -inf to 0.43603, saving model to best_model_3_v4.h5
83/83 [==============================] - 64s 680ms/step - loss: 8.0405 - accuracy: 0.3368 - val_loss: 1.8563 - val_accuracy: 0.4360
Epoch 2/30
83/83 [==============================] - ETA: 0s - loss: 2.2324 - accuracy: 0.4069
Epoch 2: val_accuracy improved from 0.43603 to 0.46296, saving model to best_model_3_v4.h5
83/83 [==============================] - 54s 650ms/step - loss: 2.2324 - accuracy: 0.4069 - val_loss: 1.8000 - val_accuracy: 0.4630
Epoch 3/30
83/83 [==============================] - ETA: 0s - loss: 2.0638 - accuracy: 0.4407
Epoch 3: val_accuracy did not improve from 0.46296
83/83 [==============================] - 52s 625ms/step - loss: 2.0638 - accuracy: 0.4407 - val_loss: 2.0296 - val_accuracy: 0.4461
Epoch 4/30
83/83 [==============================] - ETA: 0s - loss: 1.9129 - accuracy: 0.474